In [18]:
import torch
from lm_from_scratch.corpus.decision_corpus import DecisionCorpus
from lm_from_scratch.tokenizers.char_tokenizer import CharLevelTokenizer
from lm_from_scratch.models.gpt_sa import Gpt

BATCH_SIZE = 32 # how many independent sequences will we process in parallel?
MAX_LEN = 64 # what is the maximum context length for predictions?
MAX_ITERS = 10000
EVAL_INTERVAL = 1000
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200

N_EMBEDDING = 128
HEAD_SIZE = 128

# Load corpus & tokenizer

In [3]:
corpus = DecisionCorpus()

corpus_df = corpus.df.sample(
        frac=1,
        random_state=42
    ).reset_index(
        drop=True)

# Train and test splits
n = int(0.9*len(corpus_df)) # first 90% will be train, rest val

data = corpus.get_text()

tokenizer = CharLevelTokenizer("".join(data))
corpus_df["idx"] = [torch.tensor(tokenizer.encode(text), dtype=torch.long) for text in data]


train_data = corpus_df["idx"][:n]
val_data = corpus_df["idx"][n:].reset_index(drop=True)

In [4]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    doc_ix = torch.randint(len(data), (BATCH_SIZE,))
    doc_token_ix = []

    for doc_i in doc_ix:
        ix = torch.randint(len(data[int(doc_i)]) - MAX_LEN, (1,))
        doc_token_ix.append((int(doc_i), ix))
    
    x = torch.stack([data[i][j:j+MAX_LEN] for i,j in doc_token_ix])
    y = torch.stack([data[i][j+1:j+MAX_LEN+1] for i,j in doc_token_ix])

    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y


# Model training

In [19]:
model = Gpt(tokenizer.vocab_size, HEAD_SIZE, N_EMBEDDING, max_len=MAX_LEN, dropout=0.1)
m = model.to(DEVICE)

In [20]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVAL == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.9886, val loss 4.9904
step 1000: train loss 2.0966, val loss 2.1776
step 2000: train loss 2.0254, val loss 2.1127
step 3000: train loss 1.9964, val loss 2.0762
step 4000: train loss 1.9710, val loss 2.0627
step 5000: train loss 1.9645, val loss 2.0616
step 6000: train loss 1.9556, val loss 2.0565
step 7000: train loss 1.9436, val loss 2.0492
step 8000: train loss 1.9430, val loss 2.0523
step 9000: train loss 1.9433, val loss 2.0543


# Test time

In [25]:
from torch.nn import functional as F

# generate from the model
def generate(model, idx, max_new_tokens, max_len):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last max_len tokens
        idx_cond = idx[:, -max_len:]
        # get the predictions
        model.eval()
        logits, loss = model(idx_cond)
        model.train()
        
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

context = torch.tensor(
            tokenizer.encode("chambre criminelle de la Cour de cassation, com"),
            dtype=torch.long,
            device=DEVICE)

print(tokenizer.decode(generate(model,
                                context[None, :],
                                max_len=MAX_LEN,
                                max_new_tokens=10)[0].tolist()))


chambre criminelle de la Cour de cassation, comnonders 'a
